24 Oct 2020 (Ruth + Caitlin):  

1.  Identify suspected CKD
Use a combination of methods to identify records which are 'suspected CKD' and add a flag to a column in the main df.

Methods to use:
- indicator:  look at cats that have a renal food purchase
- indicator:  look at cats that have a renal medication
- indicator:  look through cats that we have extracted 'diagnosis:' for and matched renal string
- indicator:  look through cats history that has a renal string match
- mentions of renal in Examination Text

2. Eyeball records that are 'suspected CKD' by viewing the cat's entire record history to get:
- diagnosis CKD/no CKD
- date of diagnosis

IDEAS/ THINGS TO TRY FOR FINDING DIAGNOSIS

- look for renal food purchase
- look in 'Assessment' for diagnosis
- have a look at everything with renal/kidney (e.g. we don't want acute kidney disease)
- look for CKD meds e.g. Semintra in Item Label, Item Name, Examination Text
- Goal: we need to find the TIME/visit at which a cat is diagnosed renal
- PLAN:  pull everything into columns, then go through manually

In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
columns = ['Database Name', 'Patient Number', 'Client Number', 'Consultation Number', 'Item Name', 'Units',
           'Item Label', 'Clinic Code', 'Consulting Vet', 'Patient Desexed', 'Deceased Date',
           'Visit Date', 'Consultation Date', 'Examination Text' ,'Patient Species', 'Patient Breed',
           'Patient Colour', 'Patient Sex', 'Patient Date of Birth', 'Insured', 'weight', 'temperature',
           'HeartRate', 'BodyScore', 'DentalGrade', 'RespRate', 'PainScore', 'BP', 'CRT', 'MMColour']

In [3]:
vetcompass_df = pd.read_csv("../Data/vet_compass/STR024A03 20190719 sample x10,000.csv", header=None, names = columns)

/Users/ruth/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1,2,3,19,20,21,22,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
vetcompass_df.head()

,Database Name,Patient Number,Client Number,Consultation Number,Item Name,Units,Item Label,Clinic Code,Consulting Vet,Patient Desexed,Deceased Date,Visit Date,Consultation Date,Examination Text,Patient Species,Patient Breed,Patient Colour,Patient Sex,Patient Date of Birth,Insured,weight,temperature,HeartRate,BodyScore,DentalGrade,RespRate,PainScore,BP,CRT,MMColour
0,RxNSW,7138614,NaN,13869415,NaN,NaN,NaN,1,WA1,1,NaN,2012-07-05 17:09:03.000,2012-07-05 17:09:03.000,-------------------- Fees Charged: -----------...,Cat,Persian,White,Female,2010-07-05 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RxNSW,7737653,NaN,22925726,NaN,NaN,NaN,1,ZOU,1,NaN,2003-04-25 00:00:00.000,2003-04-25 00:00:00.000,NaN,Cat,Tonkinese,Lilac,Male,2002-11-26 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RxNSW,7738120,NaN,22934344,NaN,NaN,NaN,1,ZOU,1,NaN,2004-01-10 00:00:00.000,2004-01-10 00:00:00.000,NaN,Cat,Domestic Medium Hair,Tortoiseshell,Female,2003-02-14 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RxNSW,7745004,NaN,23043653,NaN,NaN,NaN,1,ZOU,1,NaN,2013-02-18 16:47:52.000,2013-02-18 16:47:52.000,NaN,Cat,Siamese,Lilac Point,Male,2008-07-20 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RxNSW,7742894,NaN,23043708,NaN,NaN,NaN,1,ZOU,1,2013-11-05 10:00:34.000,2013-02-20 08:40:38.000,2013-02-20 08:40:38.000,Dental; Renal Dz,Cat,Domestic Long Hair,Ginger & White,Male,1994-03-16 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
vetcompass_df['CKD_indicated'] = False

In [6]:
def get_patterns(df, pattern):
    # for loop of each exmination text in df as long as it is strings, re.findall: returns all matches of pattern in the string
    note_captions_lists = [re.findall(string=text, pattern=pattern) for text in df['Examination Text'] if isinstance(text, str)]
    note_captions = [text for text_list in note_captions_lists for text in text_list]
    note_captions_set = set(note_captions)
    return note_captions_set

# Look for mentions of CKD/kidney/renal

In [8]:
renal_string = "CKD.{1,150}|ckd.{1,150}|Kidney.{1,150}|kidney.{1,150}|renal.{1,150}|RENAL.{1,150}"

# try using the function
get_patterns(vetcompass_df, renal_string)

{'renal diets - last time had issues getting her to eat them; have given RC wet sachet and proplan wet renal diet to try. Collection method: Cystocentesis CU',
 'renal disease can either stay stable or worsen over time. We have also received the cytology result of the urine. They have it to be very proteinaceous wit',
 'kidneys; Feels she is drinking lots and urinating lots; Eating ok- is a fussy eater; was started on RCW Renal but with O still gets dine and old biscuits mi',
 'renal disease Ongoing: Y Date Posted: 10/05/18 Copy of Claim  attached',
 'kidney issues Vomiting twice per week Defecating - hard faeces Normal dry food now Examination: teeth - very bad - but still not sure would affect eating to',
 'renal diet as  previously',
 'renal food as used to hills brand. Have RC in stock at clinic if wants (sent sms saying wet form present for pick up). Placed on order For arthritis manage',
 'kidney; floating in ligament. Ligated vessels 3/0 monosyn and removed. Ligated uterine body

In [10]:
len(get_patterns(vetcompass_df, renal_string))

6709

In [11]:
# returns only the rows in the df where examination text includes the words in renal_string
searched_df = vetcompass_df[vetcompass_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern=renal_string)) if isinstance(text, str) else False)]

In [12]:
len(searched_df)

32808

In [13]:
len(searched_df['Patient Number'].unique())

2010

In [22]:
searched_df[['Examination Text', 'Patient Number']].iloc[100]

Examination Text    Reason: 615- R/c Appointment Notes: Overdue re...
Patient Number                                                6067233
Name: 4664, dtype: object

# Suggestion
- maybe we need to do this by hand for the 32808
- pupd meand weeing a lot, drinking a lot --> indictes CKD
- ddx means differential diagnosis
- train an NLP model to understand diagnosis?  e.g. sentiment analysis?

In [30]:
this_cat = vetcompass_df[vetcompass_df['Patient Number']==6067233]
len(this_cat)

185

In [27]:
for index, row in this_cat.iterrows():
     print(row['Examination Text'])
     print()

Reason: transfer from AEC Appointment Notes: Overdue reminders History: been to AEC overnight pcv 25 % done at pm measures blood pressure 200 so hypertensive ? Re anaemia separate problem started on norvasc 1/4 p.O. Examination: Still pale mm eaten samll amt a home hydrated admit to observe Laboratory: Assessment: Treatment: Plan: drank water quiet eaten small amt food bloods taken last night sent for urgent pcv here 27 so improved slightly pm 1/2 norvasc p.O. Keep and await reults if o ok o elected to take home will ring re path +1 29/9 GJT spoke O advised t4 normal does have azotaemia so poss anaemia + bp issues related to renal failure Advised collect urine O booked to recheck bp + collect further meds with NC on Saturday will try and collect urine to bring in then Total T4 13 nmol/L (6-52) Vetnostics North Ryde NATA Accreditation Number: 14599 Hb 80 g/L (80-140) WBC 16.7 x109/L (6.0-16.0) RCC 5.7 x1012/L (5.5-10.0) Neut 13.4 x109/L (3.8-10.1) Hct 0.24 (0.28-0.45) Lymp 2.3 x109/L (1

# Can we just look for diagnosis?

- Look for 'diagnosis' and 'differential diagnosis' 

In [ ]:
# Just search for the string 'Diagnosis:' in the examination text
# 18663 Examination Texts containging 'Diagnosis:' and of those there are 946 unique patients
# This is for ALL diagnoses, not just CKD

In [35]:
diagnosis_string = 'Diagnosis:'

In [36]:
diagnosis_df = vetcompass_df[vetcompass_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern=diagnosis_string)) if isinstance(text, str) else False)]

In [37]:
len(diagnosis_df)

18663

In [38]:
len(diagnosis_df['Patient Number'].unique())

946

In [18]:
diagnosis_df['Examination Text'].iloc[100]

'Reason: Cu Eye Appointment Notes: rw History: Owner has noticed discharge from left eye. Otherwise well. EDDU normally. No V or D. Not coughing or sneezing. No nasal discharge noted. Vaccinations UTD. Exam: Watery ocular discharge R eye and mild blepharospasm. PLR normal. Menace and palpebral normal. Conjunctiva erytheamtous and mildly swollen. Alcain drops and fluorescein stain applied. No uptake. Diagnosis: Conjunctivitis L eye Treatment: Opticin TID for 5-7 days. Discussed possibility of flare up of cat flu if had it as a young kitten and advised to ensure to be vigilant about vaccinations and reducing stress. --- 09/11/2017 09:22:14 AM AT1: Examinations - 03/11/2017 11:25:20 AM PPF Deferred: no hx Resolved - sent text  TA'

In [ ]:
#  Try using Isabel's function to look for a regex and add a new column.  
#  The new column contains all text after the regex until a colon.  Ignores case.
# Found 1634 unique values for 'diagnosis:' (diagnosis as a header)
# Found 1815 unique values for ANY mention of diagnosis (as a header or in free text)

In [39]:
def make_regex(values):
    return f'({values})[\s\:\=\-]*(.*?)\:'


def add_column(df, values, colname):
    df[colname] = df['Examination Text'].str.extract(make_regex(values), expand=True, flags=re.IGNORECASE)[1]
    return df

In [57]:
columns_to_add = {
    'diagnosis_header': 'Differential\sDiagnosis\:\sDiagnosis\:|differential\sdiagnosis\:|diagnosis\:',
    'diagnosis_mention': 'differential\sdiagnosis|diagnosis|Diagnosis',
    'history': 'history'}

In [58]:
for column_name in columns_to_add.keys():
    print("processing: "+ column_name + " with pattern " + columns_to_add[column_name])
    vetcompass_df = add_column(vetcompass_df, columns_to_add[column_name], column_name)
    print('\x1b[32m' + str(len(vetcompass_df[column_name].unique())) + " unique values found for " + column_name + '\x1b[0m')

processing: diagnosis_header with pattern Differential\sDiagnosis\:\sDiagnosis\:|differential\sdiagnosis\:|diagnosis\:
1634 unique values found for diagnosis_header
processing: diagnosis_mention with pattern differential\sdiagnosis|diagnosis|Diagnosis
1815 unique values found for diagnosis_mention
processing: history with pattern history
37756 unique values found for history


In [59]:
vetcompass_df[['Examination Text', 'diagnosis_header', 'diagnosis_mention']].dropna()['Examination Text'].iloc[7]

'Reason: Dental & Remove Growth On Lip & check lump on back & ears. Ears tend to be itchy? Skin itchy too. Is on Hills sensitive stomach & skin. Seems to agree more with her than her old diet of Iams chicken Appointment Notes: Overdue reminders History: Examination: Laboratory: Assessment: Treatment: Plan: Examination: T- P- R- OMM pink moist CRT 1-2s Pain Score:0 Laboratory: PreGA Bloods: creat 112 urea9.5 so ok Treatment: Premed: temgesic 0.15 ml scut 0.1 ml atr i/v Induction:alfaxan 1.5 ml iv via 24 g catheter Maintenance: Oxygen/isoflurane/ nitrous via 3.5 mm ET tube IV Fluids: hartmans iv Surgery:resected 2 massed nasty mass ;eft upper lip rescted skin through to mucosa sutured mucosa 4/0 monosyn continuous pattern closed skin simple interrupted also resected mass left shoulder closed scut 2/0 vicryl continuous pattern and skin 2/0 nylon simple interrupted scaled teeth send for histology both sampld to go c amoxylav 50 mgn 1 bid 10 s/o 14 days Referred by: Dr John Mackie (Auburn; 

In [61]:
vetcompass_df[['Examination Text','diagnosis_header', 'diagnosis_mention']].dropna()['diagnosis_header'].iloc[1]

'(a) Ruptured apocrine cyst with secondary inflammation (b) Apocrine sweat gland adenoma COMMENTS'

In [62]:
vetcompass_df[['Examination Text','diagnosis_header', 'diagnosis_mention']].dropna()['diagnosis_mention'].iloc[1]

'(a) Ruptured apocrine cyst with secondary inflammation (b) Apocrine sweat gland adenoma COMMENTS'

In [75]:
# Look for renal string in those that have diagnosis extracted
renal_diagnosis_df = vetcompass_df[vetcompass_df['diagnosis_mention'].apply(lambda text: bool(re.search(string=text, pattern=renal_string)) if isinstance(text, str) else False)]

In [76]:
len(renal_diagnosis_df)

508

In [78]:
len(renal_diagnosis_df['Patient Number'].unique())

55

In [80]:
renal_diagnosis_df['diagnosis'].iloc[3]

'Dental disease - requires scale & polish. May require extractions of 2 carnassials. Estimates on file. Weight loss? - (1) erronious/out of date Hx of weight recording (2) renal Dz (3) hyper-T (4) intestinal parasitism (5) other Plan'

# Look at history 

In [ ]:
# Try searching history column for renal string (history was extracted above from the Examination Text)
# Found that cats have multiple rows for single visits.  One line per item.

In [64]:
renal_history_df = vetcompass_df[vetcompass_df['history'].apply(lambda text: bool(re.search(string=text, pattern=renal_string)) if isinstance(text, str) else False)]

In [65]:
len(renal_history_df)

3879

In [66]:
len(renal_history_df['Patient Number'].unique())

321

In [67]:
renal_history_df['history'].iloc[270]

"Has been very quiet lately and possibly also did a V+ the other day. Not eating much and this morning she didn't anything. She does come for food but seems to back off her food. Also have bad teeth and CKD; OA. K/D wet food only. Also has spots on her nose and a bit weepy around the eyes. Has come and gone in the past and normally a clean up with saline settles it. Weight loss with 500 grams since last visit Skin tent slow. Sunken in eyes Cannot feel enlarged thyroid on palpation Small bladder; could not get urine Laboratory"

In [69]:
renal_history_df.iloc[270]['Patient Number']

7474421

In [72]:
renal_history_df[renal_history_df['Patient Number'] == 7474421]

,Database Name,Patient Number,Client Number,Consultation Number,Item Name,Units,Item Label,Clinic Code,Consulting Vet,Patient Desexed,Deceased Date,Visit Date,Consultation Date,Examination Text,Patient Species,Patient Breed,Patient Colour,Patient Sex,Patient Date of Birth,Insured,weight,temperature,HeartRate,BodyScore,DentalGrade,RespRate,PainScore,BP,CRT,MMColour,CKD_indicated,diagnosis,history,diagnosis_header,diagnosis_mention
118632,RxNSW,7474421,1.12045e+07,25334190,!Miscellaneous Disposables,1.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118633,RxNSW,7474421,1.12045e+07,25334190,DOC-Hosp,7.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118634,RxNSW,7474421,1.12045e+07,25334190,DOC-IV & Fluids,25.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118635,RxNSW,7474421,1.12045e+07,25334190,Electrolytes,1.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118636,RxNSW,7474421,1.12045e+07,25334190,Fluids Hartmanns Soln 1L,1.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118637,RxNSW,7474421,1.12045e+07,25334190,GX703 Free Health Check - Senior Call Campaign,1.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118638,RxNSW,7474421,1.12045e+07,25334190,GX704 $50 Off Prof Services,1.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118639,RxNSW,7474421,1.12045e+07,25334190,NUR-Hosp,20.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118640,RxNSW,7474421,1.12045e+07,25334190,NUR-IV & Fluids,20.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN,198,3/9,2,20,NaN,NaN,< 2 sec,pink,False,NaN,Has been very quiet lately and possibly also d...,NaN,NaN
118641,RxNSW,7474421,1.12045e+07,25334190,NUR-IV & Fluids Site Prep,5.0,NaN,113,KBL,1,NaN,2018-06-01 08:21:34.000,2018-06-01 08:21:34.000,Reason: Senior Check Appointment Notes: Overdu...,Cat,Domestic Short Hair,Black & White,Female,2001-07-27 00:00:00.000,NaN,2.9,NaN

In [73]:
renal_history_df[renal_history_df['Patient Number'] == 7474421]['Examination Text'].iloc[0]

"Reason: Senior Check Appointment Notes: Overdue reminders History: Has been very quiet lately and possibly also did a V+ the other day. Not eating much and this morning she didn't anything. She does come for food but seems to back off her food. Also have bad teeth and CKD; OA. K/D wet food only. Also has spots on her nose and a bit weepy around the eyes. Has come and gone in the past and normally a clean up with saline settles it. Weight loss with 500 grams since last visit Skin tent slow. Sunken in eyes Cannot feel enlarged thyroid on palpation Small bladder; could not get urine Laboratory: Inhouse PAS; electrolytes K 2.2 (3.8-5.3) BUN 26.4 (4.6-13.2) CREA 292 (62-177) PCV 30% Assessment: CKD stage 3 unmanaged Hypokalemic BP unknown - to do tomorrow before go home GHP 4 months ago showed similar values; T4 normal then Treatment: Hospitalise and IVF @ 2x maintenance (Hartmann's +20mmol KCl) Start hypokal tablets at home Plan: Lower IVF to 1.5x maint overnight BP AM AM Re-assess and an

# Try looking for renal food purchase

In [90]:
food_string = "food.{1,150}|kd.{1,150}|renal.{1,150}|k\/d.{1,150}"

In [91]:
food_df = vetcompass_df[vetcompass_df['Item Name'].apply(lambda text: bool(re.search(string=text, pattern=food_string, flags=re.IGNORECASE)) if isinstance(text, str) else False)]

In [85]:
vetcompass_df
vetcompass_df[['Item Name', 'Item Label']].sample(10)

,Item Name,Item Label
53979,RCW Dog Renal 150g (10),NaN
21014,Reminder Vax 12mth,NaN
521851,Milbemax Cat 2 - 8Kg,NaN
253258,Reminder 12mth,NaN
639690,!Miscellaneous Preventatives,NaN
84969,Anesthesia Fees,NaN
372326,RES-Exam Room,NaN
86556,Boarding,NaN
91319,!Miscellaneous Merchandise,NaN
370699,Hills Cat C/D M/Care 3.8Kg,NaN


In [98]:
food_df['Item Name'].sample(10)

355542                     RCW Cat Renal-Tuna 85 (12)
67755                RCW Cat Renal - Chicken 85g (12)
560908                            Hills Cat K/D 1.8Kg
713448                 RCW Cat Renal 85g Chicken (12)
512876                              RCW Cat Renal 4Kg
578420                    RCW Cat Renal 85g Tuna (12)
57633     Hills Cat K/D Chicken And Veg Stew 82g (24)
188037               RCW Cat Renal - Chicken 85g (12)
460622                              RCW Cat Renal 2Kg
318197                        RCW Cat Renal 100g (12)
Name: Item Name, dtype: object

In [96]:
len(food_df)

7148

In [97]:
len(food_df['Patient Number'].unique())

626

# Try medication
- phosphate binders (alutab)
- anti nausea (e.g. maxalon)
- semintra (for the treatment of protein urea in cats)
- fluid treatment 